In [1]:
"""
LICENSE MIT
2021
Guillaume Rozier
Website : http://www.covidtracker.fr
Mail : guillaume.rozier@telecomnancy.net

README:
This file contains scripts that download data from data.gouv.fr and then process it to build many graphes.
I'm currently cleaning the code, please ask me if something is not clear enough.

The charts are exported to 'charts/images/france'.
Data is download to/imported from 'data/france'.
Requirements: please see the imports below (use pip3 to install them).

"""

"\nLICENSE MIT\n2021\nGuillaume Rozier\nWebsite : http://www.covidtracker.fr\nMail : guillaume.rozier@telecomnancy.net\n\nREADME:\nThis file contains scripts that download data from data.gouv.fr and then process it to build many graphes.\nI'm currently cleaning the code, please ask me if something is not clear enough.\n\nThe charts are exported to 'charts/images/france'.\nData is download to/imported from 'data/france'.\nRequirements: please see the imports below (use pip3 to install them).\n\n"

In [2]:
import pandas as pd
import cv2
import plotly.graph_objects as go
import france_data_management as data
import plotly
PATH = '../../'
from datetime import datetime
import locale
locale.setlocale(locale.LC_ALL, 'fr_FR.UTF-8')

'fr_FR.UTF-8'

In [3]:
data.download_data()
data.download_data_hosp_fra_clage()

36it [00:06,  5.32it/s]                      


In [4]:
df_a_vacsi_a_france = data.import_data_vacsi_a_fra()
df_a_vacsi_a_france

,fra,clage_vacsi,jour,n_dose1,n_complet,n_cum_dose1,n_cum_complet,couv_dose1,couv_complet
0,FR,24,2020-12-27,1,0,1,0,0.0,0.0
1,FR,24,2020-12-28,2,0,3,0,0.0,0.0
2,FR,24,2020-12-29,0,0,3,0,0.0,0.0
3,FR,24,2020-12-30,0,0,3,0,0.0,0.0
4,FR,24,2020-12-31,1,0,4,0,0.0,0.0
...,...,...,...,...,...,...,...,...,...
1585,FR,80,2021-05-30,634,1593,3166569,2627837,76.2,63.2
1586,FR,80,2021-05-31,4893,12661,3171462,2640498,76.3,63.5
1587,FR,80,2021-06-01,7120,16550,3178582,2657048,76.4,63.9
1588,FR,80,2021-06-02,6096,14759,3184678,2671807,76.6,64.3


In [5]:
df_a_vacsi_a_france = data.import_data_vacsi_a_fra()
df_hosp_fra_clage = data.import_data_hosp_fra_clage()
clage_spf = pd.read_csv(PATH+"data/france/clage_spf.csv", sep=";")
df_a_vacsi_a_france = df_a_vacsi_a_france.merge(clage_spf, left_on="clage_vacsi", right_on="code_spf")

In [8]:
df_a_vacsi_a_france_80 = df_a_vacsi_a_france[df_a_vacsi_a_france.clage_vacsi==80]
df_hosp_fra_clage_80 = df_hosp_fra_clage[df_hosp_fra_clage.cl_age90 >= 89].groupby(["jour"]).sum().reset_index()

fig = go.Figure()
fig.add_trace(go.Scatter(
    x=df_hosp_fra_clage_80.jour,
    y=df_hosp_fra_clage_80.hosp.rolling(window=7).mean(),
    showlegend=False,
    line=dict(color="red", width=4)
))

fig.add_trace(go.Scatter(
    x=df_a_vacsi_a_france_80.jour,
    y=df_a_vacsi_a_france_80.n_cum_complet/4156974*100,
    line=dict(width=4, color="#1f77b4"),
    showlegend=False,
    yaxis="y2"
))

fig.update_layout(
    title=dict(
        y=0.90, x=0.5,
        font = dict(
                size=20, color="black"),
        text="<b>[+ de 80 ans] <span style='color:red;'>personnes hospitalisées</span> et <span style='color:#1f77b4;'>vaccinées</span></b>"),
    
    yaxis=dict(
        title="<b>Personnes hospitalisées</b>",
        titlefont=dict(
            color="red"
        ),
        tickfont=dict(
            color="red"
        )
    ),
    yaxis2=dict(
            range=[0, 100],
            title="<b>% vaccinés</b> (2 doses)",
            titlefont=dict(
                color="#1f77b4"
            ),
            ticksuffix=" %",
            tickfont=dict(
                color="#1f77b4"
            ),
            anchor="free",
            overlaying="y",
            side="right",
            position=1
        ),
    annotations = [
                dict(
                    x=0.5,
                    y=1.07,
                    xref='paper',
                    yref='paper',
                    font=dict(color="black"),
                    text='Date : {}. Données : Santé publique France. Auteur : @guillaumerozier covidtracker.fr.'.format(datetime.strptime(max(df_hosp_fra_clage_80.jour), '%Y-%m-%d').strftime('%d %B %Y')),
                    showarrow = False
                )]
)
fig.write_image(PATH + "images/charts/france/hosp_vacsi_p80.jpeg", scale=2, width=800, height=500)
plotly.offline.plot(fig, filename = PATH + 'images/html_exports/france/dc_vacsi_p80.html', auto_open=False)


'../../images/html_exports/france/dc_vacsi_p80.html'

In [9]:
df_a_vacsi_a_france_80 = df_a_vacsi_a_france[df_a_vacsi_a_france.clage_vacsi!=80].groupby(["jour"]).sum().reset_index()
df_hosp_fra_clage_80 = df_hosp_fra_clage[df_hosp_fra_clage.cl_age90 < 89].groupby(["jour"]).sum().reset_index()

fig = go.Figure()
fig.add_trace(go.Scatter(
    x=df_hosp_fra_clage_80.jour,
    y=df_hosp_fra_clage_80.hosp.rolling(window=7).mean(),
    showlegend=False,
    line=dict(color="red", width=4)
))

fig.add_trace(go.Scatter(
    x=df_a_vacsi_a_france_80.jour,
    y=df_a_vacsi_a_france_80.n_cum_complet/(66990000-4156974)*100,
    line=dict(width=4, color="#1f77b4"),
    showlegend=False,
    yaxis="y2"
))

fig.update_layout(
    title=dict(
        y=0.90, x=0.5,
        font = dict(
                size=20, color="black"),
        text="<b>[0 - 79 ans] <span style='color:red;'>personnes hospitalisées</span> et <span style='color:#1f77b4;'>vaccinées</span></b>"),
    
    yaxis=dict(
        title="<b>Personnes hospitalisées</b>",
        titlefont=dict(
            color="red"
        ),
        tickfont=dict(
            color="red"
        )
    ),
    yaxis2=dict(
            range=[0, 100],
            title="<b>% vaccinés</b> (2 doses)",
            titlefont=dict(
                color="#1f77b4"
            ),
            ticksuffix=" %",
            tickfont=dict(
                color="#1f77b4"
            ),
            anchor="free",
            overlaying="y",
            side="right",
            position=1
        ),
    annotations = [
                dict(
                    x=0.5,
                    y=1.07,
                    xref='paper',
                    yref='paper',
                    font=dict(color="black"),
                    text='Date : {}. Données : Santé publique France. Auteur : @guillaumerozier covidtracker.fr.'.format(datetime.strptime(max(df_hosp_fra_clage_80.jour), '%Y-%m-%d').strftime('%d %B %Y')),
                    showarrow = False
                )]
)
fig.write_image(PATH + "images/charts/france/hosp_vacsi_m80.jpeg", scale=2, width=800, height=500)
plotly.offline.plot(fig, filename = PATH + 'images/html_exports/france/dc_vacsi_m80.html', auto_open=False)


'../../images/html_exports/france/dc_vacsi_m80.html'

In [10]:
df_a_vacsi_a_france_80 = df_a_vacsi_a_france[df_a_vacsi_a_france.clage_vacsi==80]
df_hosp_fra_clage_80 = df_hosp_fra_clage[df_hosp_fra_clage.cl_age90 >= 89].groupby(["jour"]).sum().reset_index()

fig = go.Figure()
fig.add_trace(go.Scatter(
    x=df_hosp_fra_clage_80.jour,
    y=df_hosp_fra_clage_80.dc.diff().rolling(window=7).mean(),
    showlegend=False,
    line=dict(color="red", width=4)
))

fig.add_trace(go.Scatter(
    x=df_a_vacsi_a_france_80.jour,
    y=df_a_vacsi_a_france_80.n_cum_dose1/4156974*100,
    line=dict(width=4, color="#1f77b4"),
    showlegend=False,
    yaxis="y2"
))

fig.update_layout(
    title=dict(
        y=0.90, x=0.5,
        font = dict(
                size=20, color="black"),
        text="<b>[+ de 80 ans] <span style='color:red;'>décès hospitaliers</span> et <span style='color:#1f77b4;'>vaccinations</span></b>"),
    
    yaxis=dict(
        title="<b>Décès hospitaliers</b>",
        titlefont=dict(
            color="red"
        ),
        tickfont=dict(
            color="red"
        )
    ),
    yaxis2=dict(
            range=[0, 100],
            title="<b>% vaccinés</b> (au moins 1 dose)",
            titlefont=dict(
                color="#1f77b4"
            ),
            ticksuffix=" %",
            tickfont=dict(
                color="#1f77b4"
            ),
            anchor="free",
            overlaying="y",
            side="right",
            position=1
        ),
    annotations = [
                dict(
                    x=0.5,
                    y=1.07,
                    xref='paper',
                    yref='paper',
                    font=dict(color="black"),
                    text='Date : {}. Données : Santé publique France. Auteur : @guillaumerozier covidtracker.fr.'.format(datetime.strptime(max(df_hosp_fra_clage_80.jour), '%Y-%m-%d').strftime('%d %B %Y')),
                    showarrow = False
                )]
)
fig.write_image(PATH + "images/charts/france/dc_vacsi_p80.jpeg", scale=2, width=800, height=500)
plotly.offline.plot(fig, filename = PATH + 'images/html_exports/france/dc_vacsi_p80.html', auto_open=False)


'../../images/html_exports/france/dc_vacsi_p80.html'

In [11]:
df_a_vacsi_a_france_80 = df_a_vacsi_a_france[df_a_vacsi_a_france.clage_vacsi!=80].groupby(["jour"]).sum().reset_index()
df_hosp_fra_clage_80 = df_hosp_fra_clage[df_hosp_fra_clage.cl_age90 < 89].groupby(["jour"]).sum().reset_index()

fig = go.Figure()
fig.add_trace(go.Scatter(
    x=df_hosp_fra_clage_80.jour,
    y=df_hosp_fra_clage_80.dc.diff().rolling(window=7).mean(),
    showlegend=False,
    line=dict(color="red", width=4)
))

fig.add_trace(go.Scatter(
    x=df_a_vacsi_a_france_80.jour,
    y=df_a_vacsi_a_france_80.n_cum_dose1/(66990000-4156974)*100,
    line=dict(width=4, color="#1f77b4"),
    showlegend=False,
    yaxis="y2"
))

fig.update_layout(
    title=dict(
        y=0.90, x=0.5,
        font = dict(
                size=20, color="black"),
        text="<b>[0 - 79 ans] <span style='color:red;'>décès hospitaliers</span> et <span style='color:#1f77b4;'>vaccinations</span></b>"),
    
    yaxis=dict(
        title="<b>Décès hospitaliers</b>",
        titlefont=dict(
            color="red"
        ),
        tickfont=dict(
            color="red"
        )
    ),
    yaxis2=dict(
            range=[0, 100],
            title="<b>% vaccinés</b> (au moins 1 dose)",
            titlefont=dict(
                color="#1f77b4"
            ),
            ticksuffix=" %",
            tickfont=dict(
                color="#1f77b4"
            ),
            anchor="free",
            overlaying="y",
            side="right",
            position=1
        ),
    annotations = [
                dict(
                    x=0.5,
                    y=1.07,
                    xref='paper',
                    yref='paper',
                    font=dict(color="black"),
                    text='Date : {}. Données : Santé publique France. Auteur : @guillaumerozier covidtracker.fr.'.format(datetime.strptime(max(df_hosp_fra_clage_80.jour), '%Y-%m-%d').strftime('%d %B %Y')),
                    showarrow = False
                )]
)
fig.write_image(PATH + "images/charts/france/dc_vacsi_m80.jpeg", scale=2, width=800, height=500)
plotly.offline.plot(fig, filename = PATH + 'images/html_exports/france/dc_vacsi_m80.html', auto_open=False)


'../../images/html_exports/france/dc_vacsi_m80.html'

In [25]:
df_a_vacsi_a_france_80 = df_a_vacsi_a_france[df_a_vacsi_a_france.clage_vacsi!=80].groupby(["jour"]).sum().reset_index()
df_hosp_fra_clage_80 = df_hosp_fra_clage[df_hosp_fra_clage.cl_age90 < 89].groupby(["jour"]).sum().reset_index()

fig = go.Figure()
fig.add_trace(go.Scatter(
    x=df_hosp_fra_clage_80.jour,
    y=df_hosp_fra_clage_80.dc.diff().rolling(window=7).mean(),
    showlegend=False,
    line=dict(color="red", width=4)
))


df_a_vacsi_a_france_80 = df_a_vacsi_a_france[df_a_vacsi_a_france.clage_vacsi==80]
df_hosp_fra_clage_80 = df_hosp_fra_clage[df_hosp_fra_clage.cl_age90 >= 89].groupby(["jour"]).sum().reset_index()

fig.add_trace(go.Scatter(
    x=df_hosp_fra_clage_80.jour,
    y=df_hosp_fra_clage_80.dc.diff().rolling(window=7).mean(),
    showlegend=False,
    line=dict(color="#1f77b4", width=4)
))

fig.update_layout(
    title=dict(
        y=0.90, x=0.5,
        font = dict(
                size=20, color="black"),
        text="<b>Décès hospitaliers des <span style='color:#1f77b4;'>+ de 80 ans</span> et des <span style='color:red;'>- de 80 ans</span></b>"),
    
    yaxis=dict(
        title="<b>Décès hospitaliers</b>",
        titlefont=dict(
            color="red"
        ),
        tickfont=dict(
            color="red"
        )
    ),
    yaxis2=dict(
            range=[0, 100],
            title="<b>% vaccinés</b> (au moins 1 dose)",
            titlefont=dict(
                color="#1f77b4"
            ),
            ticksuffix=" %",
            tickfont=dict(
                color="#1f77b4"
            ),
            anchor="free",
            overlaying="y",
            side="right",
            position=1
        ),
    annotations = [
                dict(
                    x=0.5,
                    y=1.07,
                    xref='paper',
                    yref='paper',
                    font=dict(color="black"),
                    text='Date : {}. Données : Santé publique France. Auteur : @guillaumerozier covidtracker.fr.'.format(datetime.strptime(max(df_hosp_fra_clage_80.jour), '%Y-%m-%d').strftime('%d %B %Y')),
                    showarrow = False
                )]
)
fig.write_image(PATH + "images/charts/france/dc_vacsi_m80.jpeg", scale=2, width=800, height=500)
plotly.offline.plot(fig, filename = PATH + 'images/html_exports/france/dc_vacsi_m80_p80.html', auto_open=False)


'../../images/html_exports/france/dc_vacsi_m80_p80.html'

In [12]:
def dc_hosp_clage(df_hosp_fra_clage, lastday="", minday=""):    
    #lastday = df_hosp_fra_clage.jour.max()
    #lastday="2020-09-01"
    df_hosp_fra_clage_lastday = df_hosp_fra_clage[df_hosp_fra_clage.jour == lastday]
    df_hosp_fra_clage_minday = df_hosp_fra_clage[df_hosp_fra_clage.jour == minday]
    sum_hosp = df_hosp_fra_clage_lastday["hosp"].sum()

    fig = go.Figure()
    
    fig.add_trace(go.Bar(
        y=[str(age-9) + " - " + str(age) +" ans" for age in df_hosp_fra_clage_lastday["cl_age90"].values[:-1]] + ["+ 90 ans"],
        x=df_hosp_fra_clage_minday["hosp"]/df_hosp_fra_clage_minday["hosp"].sum()*100,
        marker_color='rgba(0,0,0,0)',
        marker_line_width=2,
        marker_line_color="black",
        orientation='h',
        name=minday,
        showlegend=False
    ))
    x=df_hosp_fra_clage_lastday["hosp"]/sum_hosp*100
    fig.add_trace(go.Bar(
        y=[str(age-9) + " - " + str(age) +" ans" for age in df_hosp_fra_clage_lastday["cl_age90"].values[:-1]] + ["+ 90 ans"],
        x=x,
        orientation='h',
        marker_line_width=1.5,
        marker_color="red",
        marker_line_color="red",
        text=[str(int(val)) + " %" for val in round(x)],
        textposition='auto',
        name=lastday
    ))
    value_90 = int(round((df_hosp_fra_clage_lastday["hosp"]/sum_hosp*100).values[-1]))
    
    fig.add_trace(go.Bar(
        y=[str(age-9) + " - " + str(age) +" ans" for age in df_hosp_fra_clage_lastday["cl_age90"].values[:-1]] + ["+ 90 ans"],
        x=df_hosp_fra_clage_minday["hosp"]/df_hosp_fra_clage_minday["hosp"].sum()*100,
        marker_color='rgba(0,0,0,0)',
        marker_line_width=2,
        marker_line_color="black",
        orientation='h',
        name=minday
    ))
    
    fig.update_layout(
        annotations=[
                    dict(
                        x=0.5,
                        y=1.12,
                        xref='paper',
                        yref='paper',
                        font=dict(size=11),
                        text="Lecture : les plus de 90 ans représentent {}% des personnes hospitalisées".format(value_90),
                        showarrow=False
                    ),
        ],
        legend_orientation="h",
        barmode='overlay',
        xaxis=dict(ticksuffix=" %"),
        title=dict(
            text="Part de chaque tranche d'âge dans les hospitalisations".format(lastday),
            x=0.5
        ),
        bargap=0.2
    )
    
    fig.write_image(PATH + "images/charts/france/dc_hosp_clage/{}.jpeg".format(lastday), scale=2, width=500, height=500)


In [13]:
def vacsi_clage(df_a_vacsi_a_france, lastday=""):
    #lastday = df_a_vacsi_a_france.jour.max()
    #lastday="2020-09-01"
    df_a_vacsi_a_france_lastday = df_a_vacsi_a_france[df_a_vacsi_a_france.jour == lastday].sort_values(["clage_vacsi"])
    sum_hosp = df_a_vacsi_a_france_lastday["n_cum_dose1"].sum()

    fig = go.Figure()
    x=df_a_vacsi_a_france_lastday["n_cum_dose1"]/df_a_vacsi_a_france_lastday["population"]*100
    fig.add_trace(go.Bar(
        #y=[str(age) + " ans" for age in df_a_vacsi_a_france.clage_vacsi],
        y=df_a_vacsi_a_france_lastday.categorie_fine,
        x=x,
        text=[str(int(val)) + " %" for val in round(x)],
        textposition='auto',
        orientation='h',
    ))
    value_80 = int(round((x.values[-1])))
    fig.update_layout(
         annotations=[
                    dict(
                        x=0.5,
                        y=1.12,
                        xref='paper',
                        yref='paper',
                        font=dict(size=11),
                        text="Lecture : {}% des plus de 80 ans ont reçu une dose de vaccin".format(value_80),
                        showarrow=False
                    ),
        ],
        title=dict(
            text="Couverture vaccinale {}".format(lastday),
            x=0.5
        ),
        xaxis=dict(range=[0, 100], ticksuffix=" %"),
        bargap=0
    )
    fig.write_image(PATH + "images/charts/france/vacsi_clage/{}.jpeg".format(lastday), scale=2, width=500, height=500)

In [14]:
def assemble_images(date):
    #Assemble images
    
    import numpy as np
    PATH = "../../"

    im1 = cv2.imread(PATH+'images/charts/france/vacsi_clage/{}.jpeg'.format(date))
    im2 = cv2.imread(PATH+'images/charts/france/dc_hosp_clage/{}.jpeg'.format(date))

    im_h = cv2.hconcat([im1, im2])
    cv2.imwrite(PATH+'images/charts/france/vacsi_hosp_comp/{}.jpeg'.format(date), im_h)

In [15]:
def build_video(dates):
        #import glob
    for (folder, fps) in [("vacsi_hosp_comp", 6),]:
        img_array = []
        for i in range(len(dates)):
            img = cv2.imread((PATH + "images/charts/france/{}/{}.jpeg").format(folder, dates[i]))
            height, width, layers = img.shape
            size = (width,height)
            img_array.append(img)

            if i==len(dates)-1:
                for k in range(12):
                    img_array.append(img)

            if i==0:
                for k in range(6):
                    img_array.append(img)

        out = cv2.VideoWriter(PATH + 'images/charts/france/{}.mp4'.format(folder),cv2.VideoWriter_fourcc(*'MP4V'), fps, size)

        for i in range(len(img_array)):
            out.write(img_array[i])

        out.release()

        try:
            import subprocess
            subprocess.run(["ffmpeg", "-y", "-i", PATH + "images/charts/france/{}.mp4".format(folder), PATH + "images/charts/france/{}_opti.mp4".format(folder)])
            subprocess.run(["rm", PATH + "images/charts/france/{}.mp4".format(folder)])

        except:
            print("error conversion h265")

In [16]:
days = sorted(df_a_vacsi_a_france.jour.unique())[-80:]
for date in days:
    print(date)
    vacsi_clage(df_a_vacsi_a_france, date)
    dc_hosp_clage(df_hosp_fra_clage, date, minday=days[0])
    assemble_images(date)
build_video(days)

2021-03-16
2021-03-17
2021-03-18
2021-03-19
2021-03-20
2021-03-21
2021-03-22
2021-03-23
2021-03-24
2021-03-25
2021-03-26
2021-03-27
2021-03-28
2021-03-29
2021-03-30
2021-03-31
2021-04-01
2021-04-02
2021-04-03
2021-04-04
2021-04-05
2021-04-06
2021-04-07
2021-04-08
2021-04-09
2021-04-10
2021-04-11
2021-04-12
2021-04-13
2021-04-14
2021-04-15
2021-04-16
2021-04-17
2021-04-18
2021-04-19
2021-04-20
2021-04-21
2021-04-22
2021-04-23
2021-04-24
2021-04-25
2021-04-26
2021-04-27
2021-04-28
2021-04-29
2021-04-30
2021-05-01
2021-05-02
2021-05-03
2021-05-04
2021-05-05
2021-05-06
2021-05-07
2021-05-08
2021-05-09
2021-05-10
2021-05-11
2021-05-12
2021-05-13
2021-05-14
2021-05-15
2021-05-16
2021-05-17
2021-05-18
2021-05-19
2021-05-20
2021-05-21
2021-05-22
2021-05-23
2021-05-24
2021-05-25
2021-05-26
2021-05-27
2021-05-28
2021-05-29
2021-05-30
2021-05-31
2021-06-01
2021-06-02
2021-06-03
